In [ ]:
import os.path as op
import mne
import nibabel as nib


In [ ]:
#list of subjects
subjects = []


In [ ]:
target_subject = "fsaverage"
data_path = "path"
subjects_dir = op.join(data_path, "freesurfer", "recons")

In [ ]:
for subject in subjects:
    base_subject = subject.split("_part")[0] 

    #load the stc -> the neural actvity
    stc_fname = op.join(data_path, f'path_{subject}-vl.stc')
    stc = mne.read_source_estimate(stc_fname, subject=base_subject)

    #recreate the src -> the brain grid
    surface = op.join(subjects_dir, base_subject , "bem" , "inner_skull.surf")
    src = mne.setup_volume_source_space(
        base_subject, subjects_dir=subjects_dir, pos=10.0, surface=surface, add_interpolator=True
    )


    #morph it
    morph = mne.compute_source_morph(
        src = src,
        subject_from=base_subject, 
        subject_to=target_subject,
        subjects_dir=subjects_dir, verbose=True
    )

    stc_fsaverage = morph.apply(stc)

    #save it
    stc_fsaverage.save(f"path_{subject}_to_fsaverage", overwrite=True)

    #save as a nifti file
    img_fsaverage = morph.apply(stc, mri_resolution=2, output="nifti1")
    output_path = f"path_{subject}_to_fsaverage.nii.gz"
    nib.save(img_fsaverage, output_path)